# Importanto bibliotecas

In [1]:
import ijson # $pip install ijson
import pandas as pd
import numpy as np

from nltk import word_tokenize, pos_tag, wordnet
from nltk.stem.wordnet import WordNetLemmatizer

import zenodotus as z
import noun_phrase_extractor as npe

# Importando dados de origem

In [2]:
#Leitura do DataFrame
datapath = "raw.json"

sentences = []

#Subjects
with open(datapath, 'r') as f:
    objects = ijson.items(f, 'topics.item.subject')
    sentences = list(objects)

#Questions
with open(datapath, 'r') as f:
    objects2 = ijson.items(f, 'topics.item.question')
    sentences.extend(list(objects2))

    
source = pd.DataFrame({'sentence':sentences})

# Tratamento dos dados

In [3]:
#existem dados faltantes?
source[source['sentence']==''].count()

sentence    21
dtype: int64

In [4]:
#removendo...
source = source[source['sentence']!='']
#depois da remoção
source[source['sentence']==''].count()

sentence    0
dtype: int64

In [5]:
#tokenização
source['tokenized'] = source.apply(lambda row: word_tokenize(row['sentence'].lower()), axis=1)

#Faz o tagging
source['tagged'] = source.apply(lambda row: pos_tag(row['tokenized']),axis=1)

#Lematiza somente a o token, mantendo a tag original do token antes da lematização (actors, NNS) -> (actor, NNS)
Lemmatizer = WordNetLemmatizer()
for i in source['tagged'].index:
    lista_aux = []
    for tupla in source['tagged'][i]:
        lista_aux.append( (Lemmatizer.lemmatize(tupla[0]), tupla[1]) )
    source['tagged'][i] = lista_aux

#Usa o noun-phrase extractor
source['noun-phrases'] = source.apply(lambda row: npe.extract_cleanned(row['sentence']), axis=1)

In [6]:
source.head()

,sentence,tokenized,tagged,noun-phrases
0,Bad rap actors good rap actors list?,"[bad, rap, actors, good, rap, actors, list, ?]","[(bad, JJ), (rap, NN), (actor, NNS), (good, JJ...","[[bad, rap, actor, good, rap, actor, list]]"
1,list of hispanic actors?,"[list, of, hispanic, actors, ?]","[(list, NN), (of, IN), (hispanic, JJ), (actor,...","[[hispanic, actor]]"
2,LiST OF ACTORS AND ACTRESSES?,"[list, of, actors, and, actresses, ?]","[(list, NN), (of, IN), (actor, NNS), (and, CC)...","[[list, act, ors and actresses?]]"
3,Who are a and b list actors?,"[who, are, a, and, b, list, actors, ?]","[(who, WP), (are, VBP), (a, DT), (and, CC), (b...","[[list, actor]]"
4,List of famous black actors?,"[list, of, famous, black, actors, ?]","[(list, NN), (of, IN), (famous, JJ), (black, J...","[[famous, black, actor]]"


# Acesso à knowledge base da NELL

In [7]:
#Monta um dataset com cada palavra de 'noun-phrases' em uma linha
words = []
for i,row in source.iterrows():
    for phrase in row['noun-phrases']:
        for word in phrase:
            words.append(word)
nellkb = pd.DataFrame({'word':words})

#Removendo repetições
nellkb = nellkb.drop_duplicates()

In [8]:
#Cria uma lista com cada palavra do dataset que é uma categoria da NELL

#Tenta abrir o arquivo com as categorias
try:
    with open('categories.txt', 'r') as filehandle:
        nell_categories = []
        for line in filehandle:
            # Remove '\n', que é o último caracter de cada string
            cat = line[:-1]

            # Adiciona o item à lista
            nell_categories.append(cat)
        
#Se ele não existe, cria um
except:
    #Procura cada palavra na knowledge base da NELL
    nellkb['is_category'] = nellkb.apply(lambda row: z.isCategory(row['word']),axis=1)
    nell_categories = nellkb[nellkb['is_category'] == 'yes']['word'].tolist()
    #Salva as palavras que são categoria em um arquivo
    with open('categories.txt', 'w') as filehandle:  
        for cat in categories:
            filehandle.write('%s\n' %cat)

# Gerando o Dataset

In [9]:
# Encontra quais as categorias de uma lista de palavras
def find_category(lista):
    cats = []
    for item in lista:
        if item in nell_categories:
            cats.append(item)
    return list(set(cats))

# Encontra a tag de uma determinada palavra
def get_tag(row, word):
    for tupla in row:
        if tupla[0] == word:
            retorno = tupla[1]
    #Se encontrou
    try:
        return retorno
    #Senão
    except:
        return np.nan
    
# Encontra o index de uma determinada palavra
def get_index(row, word):
    Lemmatizer = WordNetLemmatizer()
    for item in row:
        if word == Lemmatizer.lemmatize(item):
            retorno = row.index(item)
    try:
        return retorno
    except:
        return np.nan

# Calcula a distancia entre duas palavras
def distancia(row, category, word):
    cat_index = get_index(row, category)
    word_index = get_index(row, word)
    try:
        return np.abs(cat_index - word_index)
    except:
        return np.nan

In [10]:
dataset = pd.DataFrame(columns=['sentence','word','nell_category',
                                'word_tag','nell_category_tag','word_is_category',
                                'distance_from_category','number_of_occurrences','is_category_candidate'])

#Para cada linha do dataset
for irow, row in source.iterrows():
    #Pega uma das listas dentro da lista da coluna noun-phrase
    for lista in row['noun-phrases']:
        #Encontra quais são as categorias dentro da lista
        categories = find_category(lista)
        for category in categories:
            #Pega a tag da categoria
            nell_category_tag = get_tag(row['tagged'], category)
            
            #Percorre a lista palavra a palavra
            for word in lista:
                #Pega a tag da palavra
                word_tag = get_tag(row['tagged'], word)
                #Calcula a distancia entre a palavra e a categoria
                distance_from_category = distancia(row['tokenized'], category, word)
                
                #word tem que ser diferente de category para que não haja repetição do tipo actor|actor, male|male...
                if word != category:
                    if word in nell_categories:
                        word_is_category = 'yes'
                    else:
                        word_is_category = 'no'
                        
                    #Adiciona uma linha no dataset com todas as informações obtidas
                    dataset = dataset.append({'sentence':row['sentence'],
                                              'word':word,
                                              'nell_category':category,
                                              'word_tag':word_tag,
                                              'nell_category_tag':nell_category_tag,
                                              'word_is_category':word_is_category,
                                              'distance_from_category':distance_from_category},
                                              ignore_index=True)
                    
# Calcula o número de ocorrências                    
for i in range(len(dataset)):
    word = dataset['word'][i]
    nell_category = dataset['nell_category'][i]
    contador = 0
    for j in range(len(dataset)):
        if (dataset['word'][j] == word and dataset['nell_category'][j] == nell_category):
            contador += 1
    dataset['number_of_occurrences'][i] = contador

In [11]:
dataset.head()

,sentence,word,nell_category,word_tag,nell_category_tag,word_is_category,distance_from_category,number_of_occurrences,is_category_candidate
0,Bad rap actors good rap actors list?,bad,actor,JJ,NNS,no,2,3,NaN
1,Bad rap actors good rap actors list?,rap,actor,NN,NNS,no,1,2,NaN
2,Bad rap actors good rap actors list?,good,actor,JJ,NNS,no,1,11,NaN
3,Bad rap actors good rap actors list?,rap,actor,NN,NNS,no,1,2,NaN
4,Bad rap actors good rap actors list?,list,actor,NN,NNS,no,4,7,NaN


# Ainda temos que checar os valores NaN

In [12]:
dataset.isnull().sum()

sentence                    0
word                        0
nell_category               0
word_tag                   48
nell_category_tag          17
word_is_category            0
distance_from_category     54
number_of_occurrences       0
is_category_candidate     372
dtype: int64